In [2]:
import os
from keras.preprocessing import image



train_dir = 'train' 
val_dir = 'valid'
test_dir = 'test'



train_data = image.ImageDataGenerator(rescale= 1/.255) 

test_data = image.ImageDataGenerator(rescale= 1/.255)


train_gen = train_data.flow_from_directory(train_dir,target_size=(224,224),class_mode="binary")
validation_gen = test_data.flow_from_directory(val_dir , 
                                               target_size= (224,224 ),class_mode="binary")




Found 4000 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [10]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [3]:
for x, y in train_gen:
    print(x,y)
    break

[[[[133.33333    74.5098     62.7451   ]
   [125.4902     66.666664   54.901962 ]
   [125.4902     66.666664   54.901962 ]
   ...
   [274.5098    219.60785   215.68628  ]
   [254.90196   200.        196.07843  ]
   [262.7451    207.84314   203.92157  ]]

  [[129.41176    70.588234   58.82353  ]
   [125.4902     66.666664   54.901962 ]
   [125.4902     66.666664   54.901962 ]
   ...
   [278.43137   223.52942   219.60785  ]
   [262.7451    207.84314   203.92157  ]
   [266.66666   211.76471   207.84314  ]]

  [[129.41176    70.588234   58.82353  ]
   [125.4902     66.666664   54.901962 ]
   [125.4902     66.666664   54.901962 ]
   ...
   [305.88235   250.9804    247.05882  ]
   [290.19608   235.29411   231.37254  ]
   [294.11765   239.21568   235.29411  ]]

  ...

  [[964.7059    964.7059    956.86273  ]
   [964.7059    964.7059    956.86273  ]
   [964.7059    964.7059    956.86273  ]
   ...
   [968.62744   972.549     980.39215  ]
   [968.62744   972.549     980.39215  ]
   [964.7059    

In [34]:
from keras.models import Model 
from keras.layers import BatchNormalization , Dense, SeparableConv2D ,MaxPool2D , Dropout ,Input ,Flatten

inp = Input( shape=(224,224,3))
x = SeparableConv2D(32 ,3 ,activation="relu")(inp)
x = SeparableConv2D(16, 3,activation= "relu" )(x)
x = BatchNormalization(1)(x)
x = MaxPool2D(2)(x)
x = SeparableConv2D(8 , 3 ,activation="relu")(x)
x = SeparableConv2D(6 ,3 ,activation ="relu" )(x)
x = MaxPool2D(2)(x)
x = BatchNormalization(1,epsilon=1e-5)(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dense(16 , activation="relu")(x)
x = BatchNormalization()(x)
out = Dense(1,activation = "sigmoid")(x)
model = Model(inp , out)

model.compile(loss = "binary_crossentropy",optimizer = "rmsprop",metrics=["accuracy"])

In [35]:
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
separable_conv2d_44 (Separab (None, 222, 222, 32)      155       
_________________________________________________________________
separable_conv2d_45 (Separab (None, 220, 220, 16)      816       
_________________________________________________________________
batch_normalization_26 (Batc (None, 220, 220, 16)      880       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 110, 110, 16)      0         
_________________________________________________________________
separable_conv2d_46 (Separab (None, 108, 108, 8)       280       
_________________________________________________________________
separable_conv2d_47 (Separab (None, 106, 106, 6)       126 

In [36]:
history_1 = model.fit(train_gen, epochs=20  , validation_data=(validation_gen))

Epoch 1/20
125/125 [==============================] - 360s 3s/step - loss: 0.2217 - accuracy: 0.9233 - val_loss: 0.8411 - val_accuracy: 0.5000
Epoch 2/20
125/125 [==============================] - 365s 3s/step - loss: 0.1011 - accuracy: 0.9715 - val_loss: 0.8711 - val_accuracy: 0.5000
Epoch 3/20
125/125 [==============================] - 370s 3s/step - loss: 0.0499 - accuracy: 0.9865 - val_loss: 0.7880 - val_accuracy: 0.5033
Epoch 4/20
125/125 [==============================] - 336s 3s/step - loss: 0.0306 - accuracy: 0.9915 - val_loss: 0.1541 - val_accuracy: 0.9600
Epoch 5/20
125/125 [==============================] - 307s 2s/step - loss: 0.0252 - accuracy: 0.9915 - val_loss: 0.2443 - val_accuracy: 0.9433
Epoch 6/20
125/125 [==============================] - 305s 2s/step - loss: 0.0158 - accuracy: 0.9950 - val_loss: 0.6044 - val_accuracy: 0.8000
Epoch 7/20
125/125 [==============================] - 305s 2s/step - loss: 0.0202 - accuracy: 0.9930 - val_loss: 0.2145 - val_accuracy: 0.9567

In [38]:
model.save("A&B_class_Val_acc97.h5")

In [39]:
test_gen = test_data.flow_from_directory("test",target_size=(224,224),class_mode='binary')


testing = model.evaluate(test_gen)

Found 300 images belonging to 2 classes.


In [42]:
print("Testing  accuracy is " , testing[1], "with loss ", testing[0])

Testing  accuracy is  0.99 with loss  0.023090877003050993


In [3]:
from keras.preprocessing import image
import numpy as np

In [13]:
from keras.models import load_model
l_model = load_model("A&B_class_Val_acc97.h5")
img = image.load_img("b.png",target_size=(224,224))
img = image.img_to_array(img )
pic  = img * 1./255
pic.dtype


dtype('float32')

In [18]:
pic=pic.reshape((1,224,224,3))

In [19]:
label = ["Beautiful"]
pred = l_model.predict(pic)
if pred < 0.5 :
    Print(f"Actual labels is {label} and predicted label is Average")
else :
    print(f"Actual labels is {label} and predicted label is Beautiful")

Actual labels is ['Beautiful'] and predicted label is Beautiful
